# 演習
より性能の高い、CNNのモデルを構築しましょう。  
本セクションで構築したモデルを改良し、テストデータの正解率80%以上を目指しましょう。



## ●データセットの読み込みと前処理
今回も、cifar-10を使用します。  
以下のセルの、バッチサイズとエポック数は変更しても構いません。

In [ ]:
import numpy as np
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

(x_train, t_train), (x_test, t_test) = cifar10.load_data()

batch_size = 32
epochs = 20
n_class = 10

x_train = x_train / 255  # 0から1の範囲に収める
x_test = x_test / 255
t_train = keras.utils.to_categorical(t_train, n_class)  # one-hot表現に
t_test = keras.utils.to_categorical(t_test, n_class)

以下のモデルは、本セクションで構築したものと同じです。  
層の追加や削除、ハイパーパラメータの調整、ドロップアウトの挿入などを行い、このモデルに改良を加ましょう。

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))  # ゼロパディング、バッチサイズ以外の画像の形状を指定
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # 一次元の配列に変換
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))  # ドロップアウト
model.add(Dense(n_class))
model.add(Activation('softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## ●学習
ImageDataGeneratorの設定は、データ拡張のレクチャーの際と同じです。  
こちらの設定にも、改良を加ましょう。  
学習には時間がかかりますので、編集→ノートブックの設定のハードウェアアクセラレーターでGPUを選択しましょう。

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

generator = ImageDataGenerator(
           rotation_range=0.2,
           horizontal_flip=True)
generator.fit(x_train)

history = model.fit_generator(generator.flow(x_train, t_train, batch_size=batch_size),
                              epochs=epochs,
                              validation_data=(x_test, t_test))

## ●学習の推移

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']  # 訓練用データの誤差
train_acc = history.history['accuracy']  # 訓練用データの精度
val_loss = history.history['val_loss']  # 検証用データの誤差
val_acc = history.history['val_accuracy']  # 検証用データの精度

plt.plot(np.arange(len(train_loss)), train_loss, label='loss')
plt.plot(np.arange(len(val_loss)), val_loss, label='val_loss')
plt.legend()
plt.show()

plt.plot(np.arange(len(train_acc)), train_acc, label='acc')
plt.plot(np.arange(len(val_acc)), val_acc, label='val_acc')
plt.legend()
plt.show()

## ●評価
こちらの正解率が0.8を超えるように、モデルや学習方法を改良しましょう。

In [ ]:
loss, accuracy = model.evaluate(x_test, t_test)
print("誤差:", loss, "正解率:", accuracy)

今回は解答例はありません。  
例え正解率が0.8に届かなくても、モデルや条件に様々な工夫をされたのであればそれで十分です。